**Parcours Ingénieur Machine Learning**<br>
**Plus d'informations** : https://openclassrooms.com/fr/paths/148-ingenieur-machine-learning <br>

**Auteur** : Viktoriya Zeruk<br>
**Date dernière version** : 08/08/2022<br>
**Accès projet git** : https://github.com/viczer/P7-Openclassroom-preuve-de-concept

---


<div style="display: flex; background: rgb(75,0,130);
background: linear-gradient(90deg, rgba(75,0,130,1) 47%, rgba(216,191,216,1) 89%, rgba(230,230,250,1) 100%);">
<h2 style="margin: auto; font-weight: bold; padding: 30px 30px 0px 30px;" align="center">| Project 7 : Développez une preuve de concept | <br></h2> </div>    

<div style="display: flex; background: rgb(75,0,130);
background: linear-gradient(90deg, rgba(75,0,130,1) 47%, rgba(216,191,216,1) 89%, rgba(230,230,250,1) 100%);">
<h4 style="margin: auto; font-weight: bold; padding: 30px 30px 0px 30px;" align="center"> </h4> 
</div>    

---
# 1. Contexte

Lors d'un précédent projet (Project 6 : Classez des images à l'aide d'algorithmes de Deep Learning), nous avions utilisé le réseau de neurones convolutif.
Le meilleur modèle du projet précédent : *Xception*

## Les données
 Stanford Dogs Dataset

## Mission
Tester une nouvelle méthode ViT (Vision Transformer) concurrente au  méthode CNN. 
L'idée est donc de comparer, en termes de précision et temps de calcul, un CNN et un ViT. Pour ce faire, nous réutiliserons le Stanford Dogs Dataset et le modèle Xception précédemment entraîné que l'on comparera au modèle ViT-B/16 de Google.

## Ressources de calcul
L'entraînement (même partiel) d'un réseau de neurones convolutionnels est très gourmand en ressources.
 Solutions :

* Limiter le jeu de données, en ne sélectionnant que quelques classes (races de chiens), ce qui permettra déjà de tester la démarche et la conception des modèles, avant une éventuelle généralisation.
* Utiliser la carte graphique de l’ordinateur en tant que GPU (l'installation est un peu fastidieuse, et l'ordinateur est inutilisable le temps du calcul).

# Sommaire

* <a href="#C1">I - Importation des données</a>
* <a href="#C2">II - Méthodes de classification d'images</a>
    * <a href="#C3">Xception</a>
    * <a href="#C4">Vision Transformer</a>
* <a href="#C5">III - Conclusion</a>

# <a name="C1">I - Importation des données</a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
import timeit
import torch
import pytorch_lightning as pl

from sklearn.model_selection import train_test_split
from PIL import Image
from matplotlib.pyplot import imread
from datasets.dataset_dict import DatasetDict
from datasets import Dataset, load_dataset, load_metric

from keras.preprocessing import image
from keras.applications.xception import Xception
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D, Dense

from transformers import ViTFeatureExtractor, ViTForImageClassification, TrainingArguments, Trainer

In [6]:
%%time

#On récupère les uri de chaque image
dogs_lst = []
for file in glob.glob('../input/images/images/*/*.jpg'):
    dogs_lst.append(file)

#On les stocke dans un dataframe
data = pd.DataFrame(dogs_lst, columns=['uri'])

#On récupère la race du chien
data['breed'] = data['uri'].apply(lambda x: x.split('/')[-2].split('-')[-1])

#On récupère le nombre de canaux utilisés
data['nb_color'] = data['uri'].apply(lambda x: imread(x).shape[2])

data.head()

CPU times: user 1min 6s, sys: 2.34 s, total: 1min 8s
Wall time: 2min 28s


,uri,breed,nb_color
0,../input/stanford-dogs-dataset/images/Images/n...,otterhound,3
1,../input/stanford-dogs-dataset/images/Images/n...,otterhound,3
2,../input/stanford-dogs-dataset/images/Images/n...,otterhound,3
3,../input/stanford-dogs-dataset/images/Images/n...,otterhound,3
4,../input/stanford-dogs-dataset/images/Images/n...,otterhound,3


In [10]:
#Exportation du jeu de donées
data.to_csv('data.csv', sep = ',', encoding='utf-8', index=False)

In [3]:
#Chargement du jeu de données
data = pd.read_csv('../input/data/data.csv', sep=',',encoding='utf-8')
data.head()

,uri,breed,nb_color
0,../input/stanford-dogs-dataset/images/Images/n...,otterhound,3
1,../input/stanford-dogs-dataset/images/Images/n...,otterhound,3
2,../input/stanford-dogs-dataset/images/Images/n...,otterhound,3
3,../input/stanford-dogs-dataset/images/Images/n...,otterhound,3
4,../input/stanford-dogs-dataset/images/Images/n...,otterhound,3


Une première chose à vérifier est la présence éventuelle de valeurs manquantes.

In [4]:
data.isnull().sum()

uri         0
breed       0
nb_color    0
dtype: int64

In [9]:
data['nb_color'].value_counts()

3    20579
4        1
Name: nb_color, dtype: int64

Une des photos dans le jeu de données comporte, en plus des trois couches RGB, une couche Alpha, qui permet de coder la transparence de l'image. Toutefois, cette couche n'est souvent pas compatibles avec les modèles de traitement d'image. On va donc la retirer pour éviter de potentiels soucis.

In [13]:
idx = data[data['nb_color']==4].index[0]
data.drop(data.index[idx], axis=0, inplace=True)
data['nb_color'].value_counts()

3    20579
Name: nb_color, dtype: int64

In [14]:
nb_pictures = len(data)
nb_breeds = len(data['breed'].unique())
print(f"Le jeu de données comporte {nb_pictures} photos de chiens, représentant {nb_breeds} races différentes")

Le jeu de données comporte 20579 photos de chiens, représentant 119 races différentes


# <a name="C2">II -  Méthodes de classification d'images</a>

Notre problématique est de trouver une méthode de classification d'images de chiens en fonction de la race. Nous comparerons pour ce faire une méthode CNN et une méthode Vision Transformer. La comparaison se fera grâce au score Accuracy et au temps de calcul.</br>
Commençons par séparer notre jeu de données en jeu d'entraînement et jeu de test de sorte à entraîner les modèles à partir des mêmes données.

In [15]:
#Séparation jeux entraînement/test
train, test = train_test_split(data, test_size=0.2, shuffle=True, random_state=42)

#Exportation des jeux d'entraînement et de test
train.to_csv('train.csv', sep = ',', encoding='utf-8', index=False)
test.to_csv('test.csv', sep=',', encoding='utf-8', index=False)

In [3]:
#Chargement du jeu d'entraînement
train = pd.read_csv('../input/data/train.csv', sep=',',encoding='utf-8')

#Chargement du jeu de test
test = pd.read_csv('../input/data/test.csv', sep=',',encoding='utf-8')

In [23]:
#Tableau de comparaison des modèles
df_res = pd.DataFrame([], index=['Xception', 'Vit'], columns=['Temps entraînement', 'accuracy train', 'accuracy test'])
df_res

,Temps entraînement,accuracy train,accuracy test
Xception,NaN,NaN,NaN
Vit,NaN,NaN,NaN


## <a name="C3">Xception</a>

Le modèle Xception est un réseau de neurones convolutifs profond développé en 2017 par Google. Sa principale caractéristique est qu'il dispose de couches de convolution profondes, alternatives aux couches de convolution classiques qui ont pour but de réduire les temps de calcul. Il est également constitué de connexions récurrentes, c'est-à-dire des boucles de rétropropagation qui permettent de conserver en mémoire des informations obtenues lors d'étapes précédentes et de les utiliser au moment de prendre des décisions dans les étapes suivantes.</br>
<img src="https://www.researchgate.net/publication/343535666/figure/fig2/AS:930842713014272@1598941605167/Architecture-of-Xception-model-obtained-from-Chollet-2017.png"> <br>
Nous l'entraînerons en utilisant un fine-tuning partiel, en ré-entraînant 10% des couches convolutionnelles hautes.

In [4]:
def prepare_data_cnn(train, test):
    """Fonction qui prend en entrée le train et le test et renvoie les données préparées pour le modèle Xception"""
    
    #On redimensionne les images
    datagen = image.ImageDataGenerator(rescale=1./255)
    train_gen = datagen.flow_from_dataframe(train, 
                                            x_col='uri', 
                                            y_col='breed', 
                                            target_size=(224, 224),
                                            seed=42)
    test_gen = datagen.flow_from_dataframe(test, 
                                            x_col='uri', 
                                            y_col='breed', 
                                            target_size=(224, 224),
                                            seed=42)

    return (train_gen, test_gen)

In [19]:
def cnn_fine_tuning(nb_breeds):
    """Fonction qui prend en entrée le nombre de races utilisées
    et renvoie le modèle Xception prêt pour le transfer learning avec fine-tuning partiel"""

    #Charger le modèle pré-entraîné sans les couches fully-connected
    model = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    #Entraîner 10% des couches hautes
    nb_10 = int(len(model.layers)*0.9)
    for layer in model.layers[:nb_10]:
        layer.trainable = False
    for layer in model.layers[nb_10:]:
        layer.trainable = True

    #Définir le nouveau modèle
    new_model = Sequential()
    new_model.add(model)
    new_model.add(GlobalAveragePooling2D())
    new_model.add(Dense(256, activation='relu'))
    new_model.add(Dense(nb_breeds, activation='softmax'))

    return new_model

In [5]:
%%time
train_cnn, test_cnn = prepare_data_cnn(train, test)
breeds_dict = (train_cnn.class_indices)

Found 16463 validated image filenames belonging to 119 classes.
Found 4116 validated image filenames belonging to 119 classes.
CPU times: user 639 ms, sys: 738 ms, total: 1.38 s
Wall time: 36 s


In [25]:
#Création et entrainement du modèle
start_time = timeit.default_timer()

cnn = cnn_fine_tuning(nb_breeds)
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cnn_res = cnn.fit(train_cnn, epochs=50, verbose=0)

time = timeit.default_timer() - start_time

2022-07-13 12:20:10.759083: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-07-13 12:20:14.843585: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


In [26]:
#On récupère les scores
df_res.loc['Xception', 'Temps entraînement'] = time
df_res.loc['Xception', 'accuracy train'] = cnn_res.history['accuracy'][-1]
df_res.loc['Xception', 'accuracy test'] = cnn.evaluate(test_cnn)[1]
df_res

129/129 [==============================] - 27s 200ms/step - loss: 2.0960 - accuracy: 0.7272


,Temps entraînement,accuracy train,accuracy test
Xception,4462.854993,0.987791,0.727162
Vit,NaN,NaN,NaN


## <a name="C4">Vision Transformer</a>

Un Vision Transformer n'est autre que l'application d'un Transformer, couramment utilisé en traitement du langage, pour le traitement d'images. Les Transformers mesurent les relations entre les paires de tokens d'entrée (mots dans le cas de chaînes de texte), appelées attention. Le coût est quadratique par rapport au nombre de tokens. Pour les images, l'unité de base de l'analyse est le pixel. Cependant, le calcul des relations pour chaque paire de pixels dans une image typique est prohibitif en termes de mémoire et de calcul. Au lieu de cela, ViT calcule les relations entre les pixels dans diverses petites sections de l'image (par exemple, 16x16 pixels pour le modèle ViT-B/16 que l'on va utiliser), à un coût considérablement réduit. Les sections (avec les indices de position) sont placées dans une séquence, puis réduites linéairement afin de réduire la taille. Les séquences/tokens ainsi obtenues peuvent alors être passées en entrée d'un Transformer pour la classification.<br>
<img src="https://viso.ai/wp-content/uploads/2021/09/vision-transformer-vit.png"> <br>

Les ViT peuvent donner de meilleurs résultats que les CNN à condition d'avoir été entraînés sur des jeux de données très conséquent. Le modèle ViT-B/16 utilisé à été pré-entraîné sur le jeu de données ImageNet-21k contenant plus de 14 millions d'images. Notre jeu de données étant petit, on va utiliser une méthode de fine-tuning sur le modèle pré-entraîné disponible sur la plateforme HuggingFace.

In [27]:
def get_breed_num(breed):
    """Fonction qui renvoie le numéro correspondant à la race passée en argument"""
    return breeds_dict[breed]

In [28]:
%%time
train_vit = train.copy()
test_vit = test.copy()

train_vit['breed_num'] = train_vit['breed'].apply(lambda x: get_breed_num(x))

test_vit['breed_num'] = test_vit['breed'].apply(lambda x: get_breed_num(x))

CPU times: user 17.1 ms, sys: 2.05 ms, total: 19.2 ms
Wall time: 18.3 ms


In [29]:
#Exportation des jeux d'entraînement et de test
train_vit.to_csv('train_vit.csv', sep = ',', encoding='utf-8', index=False)
test_vit.to_csv('test_vit.csv', sep=',', encoding='utf-8', index=False)

In [6]:
#Chargement des jeux d'entraînement et de test dans un dataset Hugging Face
dataset = load_dataset('csv', data_files={'train': '../input/data/train_vit.csv', 'test': '../input/data/test_vit.csv'})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-cf13d0b5eea460fb/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [31]:
dataset

DatasetDict({
    train: Dataset({
        features: ['uri', 'breed', 'nb_color', 'breed_num'],
        num_rows: 16463
    })
    test: Dataset({
        features: ['uri', 'breed', 'nb_color', 'breed_num'],
        num_rows: 4116
    })
})

Lorsque les modèles ViT sont entraînés, des transformations spécifiques sont appliquées aux images qui leur sont fournies. Pour s'assurer que l'on applique les bonnes transformations, nous utiliserons un ViTFeatureExtractor initialisé avec une configuration qui a été sauvegardée avec le modèle pré-entraîné que nous prévoyons d'utiliser. Dans notre cas, nous utiliserons le modèle google/vit-base-patch16-224-in21k et chargerons son extracteur de caractéristiques.

In [7]:
model_name = 'google/vit-base-patch16-224-in21k'
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)

Downloading:   0%|          | 0.00/160 [00:00<?, ?B/s]

In [8]:
def transform(example_batch):
    """Fonction en entrée un DictDataset des images et les prépare pour le modèle ViT"""
    
    inputs = feature_extractor([Image.open(x) for x in example_batch['uri']], return_tensors='pt')
    inputs['labels'] = example_batch['breed_num']
    return inputs

prepared_ds = dataset.with_transform(transform)

In [9]:
def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

In [10]:
metric = load_metric("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

On peut à présent télécharger le modèle en veillant à bien préciser le nombre de classes utilisées.

In [11]:
model = ViTForImageClassification.from_pretrained(
    model_name,
    num_labels=len(breeds_dict),
    id2label={v: k for v, k in enumerate(breeds_dict)},
    label2id=breeds_dict
)

Downloading:   0%|          | 0.00/502 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/330M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
training_args = TrainingArguments(
  output_dir="./vit-base-beans-demo-v5",
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=4,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["test"],
    tokenizer=feature_extractor,
)

Using amp half precision backend


In [14]:
start_time = timeit.default_timer()

train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

time = timeit.default_timer() - start_time

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 16463
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 4116


Step,Training Loss,Validation Loss,Accuracy
100,3.785700,3.747199,0.539845
200,2.990100,2.946486,0.611273
300,2.261000,2.304302,0.631195
400,1.900800,1.876223,0.668367
500,1.661000,1.615720,0.683431
600,1.477500,1.435463,0.682945
700,1.309900,1.243238,0.713557
800,1.139600,1.205417,0.714529
900,0.984500,1.066272,0.735180
1000,1.098400,1.032939,0.732507


***** Running Evaluation *****
  Num examples = 4116
  Batch size = 8
Saving model checkpoint to ./vit-base-beans-demo-v5/checkpoint-100
Configuration saved in ./vit-base-beans-demo-v5/checkpoint-100/config.json
Model weights saved in ./vit-base-beans-demo-v5/checkpoint-100/pytorch_model.bin
Feature extractor saved in ./vit-base-beans-demo-v5/checkpoint-100/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 4116
  Batch size = 8
Saving model checkpoint to ./vit-base-beans-demo-v5/checkpoint-200
Configuration saved in ./vit-base-beans-demo-v5/checkpoint-200/config.json
Model weights saved in ./vit-base-beans-demo-v5/checkpoint-200/pytorch_model.bin
Feature extractor saved in ./vit-base-beans-demo-v5/checkpoint-200/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 4116
  Batch size = 8
Saving model checkpoint to ./vit-base-beans-demo-v5/checkpoint-300
Configuration saved in ./vit-base-beans-demo-v5/checkpoint-300/config.json
Model weights save

***** train metrics *****
  epoch                    =          4.0
  total_flos               = 4757525103GF
  train_loss               =       0.7147
  train_runtime            =   1:16:05.06
  train_samples_per_second =       14.425
  train_steps_per_second   =        0.902


In [15]:
print(time)

4566.369979034001


In [16]:
metrics = trainer.evaluate(prepared_ds['test'])
trainer.log_metrics("eval", metrics)

***** Running Evaluation *****
  Num examples = 4116
  Batch size = 8


***** eval metrics *****
  epoch                   =        4.0
  eval_accuracy           =     0.8243
  eval_loss               =     0.6968
  eval_runtime            = 0:00:59.40
  eval_samples_per_second =     69.288
  eval_steps_per_second   =      8.669


In [17]:
metrics = trainer.evaluate(prepared_ds['train'])
trainer.log_metrics("eval", metrics)

***** Running Evaluation *****
  Num examples = 16463
  Batch size = 8


***** eval metrics *****
  epoch                   =        4.0
  eval_accuracy           =     0.9922
  eval_loss               =     0.0388
  eval_runtime            = 0:05:02.58
  eval_samples_per_second =     54.408
  eval_steps_per_second   =      6.801


In [26]:
#On récupère les scores
df_res.loc['Vit', 'Temps entraînement'] = time
df_res.loc['Vit', 'accuracy train'] = 0.9922 
df_res.loc['Vit', 'accuracy test'] = 0.8243
df_res

,Temps entraînement,accuracy train,accuracy test
Xception,4462.854993,0.987791,0.727162
Vit,4566.369979,0.9922,0.8243


# <a name="C5">III - Conclusion</a>

Le modèle ViT utilise l'auto-attention multi-têtes en vision par ordinateur sans nécessiter les biais spécifiques à l'image. Le modèle divise les images en une série de patchs d'intégration positionnels, qui sont traités par l'encodeur du transformateur. Il le fait pour comprendre les caractéristiques locales et globales que possède l'image. Enfin, le ViT a un taux de précision plus élevé sur un grand ensemble de données avec un temps de formation réduit.
 
L’inconvénient d’un ViT est qu’il nécessite plus de données d’entraînement qu’un CNN.

Les deux modèles ont des temps de calcul similaires mais un score de précision significativement meilleur pour le ViT. Le modèle ViT est toutefois plus compliqué à mettre en oeuvre, notamment si l'on souhaite optimiser les hyperparamètres ou ajouter de la Data Augmentation. 
